# Robinhood Tracking
> **Goal**: Captures fundamentals on provided companies. <br>
> **Notebook Owner(s):** Time Bioventures <br>
> **Date:** July 2024 <br>

***
## Import Libraries

Import the libraries you will be using throughout the project.

In [1]:
# Imports
import os
import sys
import json
import numpy as np
import pandas as pd
from pprint import pprint
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import helper.login_helper as login_helper

rs_installed = login_helper.login_helper()

ERROR: There was an issue loading pickle file. Authentication may be expired - logging in normally.
robin_stocks imported


***
## Holdings
`create_holdings_df()` captures all current holdings and inserts into a pandas DataFrame for viewing

In [2]:
import pytz
import robin_stocks.robinhood as rs
from collections import defaultdict

def create_holdings_df():
	'''
	create_holdings_df requests all holdings information from 
	robinhood, builds a pandas DataFrame to capture all the 
	data, and returns the DataFrame
	'''
	nytz = pytz.timezone('America/New_York')
	holding_dict = defaultdict(str)
	my_holdings = rs.build_holdings()
	holdings_df = pd.DataFrame(my_holdings).T
	cols = list(holdings_df.columns.values)
	cols = ['name']  + [col for col in holdings_df if col != 'name']
	holdings_df = holdings_df[cols]
	return holdings_df

holdings_df = create_holdings_df()
holdings_df

,name,price,quantity,average_buy_price,equity,percent_change,intraday_percent_change,equity_change,type,id,pe_ratio,percentage
SPY,SPDR S&P 500 ETF,546.520000,1.56288200,405.4753,854.15,34.79,-0.65,220.436223,etp,8f92e76f-1e0e-4478-8580-16a6ffcfaef5,25.940980,5.75
GOOG,Alphabet Class C,173.050000,16.01796000,97.9270,2771.91,76.71,0.00,1203.317209,stock,943c5009-a0bb-4665-8cf4-a95dab5874e4,24.836800,18.67
AAPL,Apple,220.280000,22.79212100,128.9358,5020.65,70.84,0.00,2081.928059,stock,450dfc6d-5510-4d40-abfb-f633b7d9be3e,34.533800,33.81
ENPH,Enphase Energy,113.000000,0.75363200,159.9557,85.16,-29.36,0.00,-35.387318,stock,69cbc174-ead1-49a9-9c72-a8f015de474f,125.803000,0.57
ALVR,AlloVir,0.740000,0.63708000,15.6151,0.47,-95.26,0.00,-9.476629,stock,fde8cc68-e63a-4a3a-b97b-897a99d203f9,-0.450000,0.00
CMPS,Compass Pathways,7.140000,6.23831500,27.1493,44.54,-73.70,0.00,-124.824316,adr,933c11d8-70dc-482d-8116-8d879c9239cf,-3.850000,0.30
AMZN,Amazon,187.060000,5.00000000,150.7540,935.30,24.08,0.00,181.530000,stock,c0bb3aec-bd1e-471e-a4f0-ca011cbec711,52.513600,6.30
DNA,Ginkgo Bioworks,0.381200,40.89477700,12.2265,15.59,-96.88,0.00,-484.410902,stock,bf0e8f54-791a-4237-a034-934db7fbd219,-0.870000,0.10
LEU,Centrus Energy,40.810000,47.71139900,43.7338,1947.10,-6.69,0.00,-139.498588,stock,55da1c15-28c0-47f4-ada6-e0fcb3f9cc9e,9.610010,13.11
MVIS,MicroVision,1.015000,195.00000000,6.1919,197.92,-83.61,0.00,-1009.495500,stock,0d44bf53-5511-4269-99bc-8c73029db529,-2.240000,1.33


***
## Order History

`create_orders_df()` captures all the stock orders (both buys and sells) executed on Robinhood, and inserts it into a pandas DataFrame for viewing

In [11]:
def create_orders_df():
	nytz = pytz.timezone('America/New_York')
	order_dict = defaultdict(str)
	orders_dict_list = []
	my_orders = rs.get_all_stock_orders()
	for order in my_orders:
		if order['state'] == 'cancelled':
			continue
		stock_url = order['instrument']
		stock_info = rs.stocks.get_instrument_by_url(stock_url)
		order_dict['ticker'] = stock_info['symbol']
		order_dict['name'] = stock_info['simple_name']
		try:
			order_dict['avg_price'] = float(order['average_price'])
		except:
			order_dict['avg_price'] = 0 # received as gift or free stock
		try:
			order_dict['latest_price'] = float(rs.stocks.get_latest_price(order_dict['ticker'])[0])
			order_dict['quantity'] = float(order['cumulative_quantity'])
			order_dict['total_amt'] = order_dict['avg_price'] * order_dict['quantity']
			date_time = datetime.strptime(order['created_at'],
							'%Y-%m-%dT%H:%M:%S.%fZ').astimezone(nytz)
			order_dict['current_val'] = order_dict['latest_price']*order_dict['quantity'] 
			order_dict['date'] = date_time.date()
			order_dict['time'] = date_time.time()
			order_dict['order_type'] = order['side']
			order_df = pd.DataFrame(order_dict, index=[0])
			orders_dict_list.append(order_df)
		except:
			pass
		
	orders_df = pd.concat(orders_dict_list, ignore_index=True)
	return orders_df
	
orders_df = create_orders_df()
orders_df

400 Client Error: Bad Request for url: https://api.robinhood.com/quotes/?symbols=TWTR
400 Client Error: Bad Request for url: https://api.robinhood.com/quotes/?symbols=RUBY
400 Client Error: Bad Request for url: https://api.robinhood.com/quotes/?symbols=NAKD
400 Client Error: Bad Request for url: https://api.robinhood.com/quotes/?symbols=NAKD
400 Client Error: Bad Request for url: https://api.robinhood.com/quotes/?symbols=RUBY
400 Client Error: Bad Request for url: https://api.robinhood.com/quotes/?symbols=RUBY


,ticker,name,avg_price,latest_price,quantity,total_amt,current_val,date,time,order_type
0,CX,Cemex,6.3400,6.8550,0.326498,2.069997,2.238144,2024-06-27,01:08:23.272172,buy
1,GOOG,Alphabet Class C,178.1737,189.2400,0.017960,3.200000,3.398750,2024-06-18,01:48:36.659694,buy
2,X,United States Steel,36.4001,39.8600,0.048901,1.780001,1.949194,2024-06-13,00:43:00.830651,buy
3,AAPL,Apple,190.2501,235.6397,0.029908,5.690000,7.047512,2024-05-17,01:34:23.687418,buy
4,SPY,SPDR S&P 500 ETF,500.9089,563.9601,0.004951,2.480000,2.792166,2024-05-01,00:58:02.605489,buy
...,...,...,...,...,...,...,...,...,...,...
143,NTLA,Intellia Therapeutics,29.1100,26.2400,10.000000,291.100000,262.400000,2018-07-17,00:58:06.581995,buy
144,EDIT,Editas Medicine,36.3200,5.5150,8.000000,290.560000,44.120000,2018-06-14,13:37:26.599966,sell
145,NTLA,Intellia Therapeutics,25.9100,26.2400,5.000000,129.550000,131.200000,2018-06-14,13:10:37.245826,buy
146,EDIT,Editas Medicine,36.1675,5.5150,8.000000,289.340000,44.120000,2018-06-12,14:35:02.106426,buy


***
## Search Company Info

In [31]:
selected_company = 'AGIOS'
company_info = rs.find_instrument_data(selected_company)
company_info_df = pd.DataFrame(company_info)
company_info_df

Found 1 results


,id,url,quote,fundamentals,splits,state,market,simple_name,name,tradeable,...,internal_halt_sessions,internal_halt_start_time,internal_halt_end_time,internal_halt_source,all_day_tradability,notional_estimated_quantity_decimals,tax_security_type,reserved_buying_power_percent_queued,reserved_buying_power_percent_immediate,otc_market_tier
0,c2186355-945c-48c6-9b1c-0df7dbd9d957,https://api.robinhood.com/instruments/c2186355...,https://api.robinhood.com/quotes/AGIO/,https://api.robinhood.com/fundamentals/AGIO/,https://api.robinhood.com/instruments/c2186355...,active,https://api.robinhood.com/markets/XNAS/,Agios,"Agios Pharmaceuticals, Inc. Common Stock",True,...,None,None,None,,untradable,5,stock,0.10000000,0.05000000,


### Company Fundamentals

In [30]:
company_symbol = company_info[0]['symbol']
# get market cap and sector
company_fundamentals = rs.stocks.get_fundamentals(company_symbol)
# show as pandas
company_fundamentals_df = pd.DataFrame(company_fundamentals)
company_fundamentals_df

,open,high,low,volume,overnight_volume,bounds,market_date,average_volume_2_weeks,average_volume,average_volume_30_days,...,headquarters_state,sector,industry,num_employees,year_founded,payable_date,ex_dividend_date,financial_status_indicator,financial_status_description,symbol
0,46.300000,46.490000,45.240000,95558.000000,0.000000,regular,2024-07-15,604622.700000,604622.700000,722077.150000,...,Massachusetts,Health Technology,Pharmaceuticals: Major,383,2007,None,None,CC0,,AGIO


In [45]:
company_list = ['Pfizer', 'Amgen', 'Novartis', 'Eli Lilly', 'Gilead', 'Abbvie', 'Biogen', 'Regeneron', 'Vertex', 'Agios']
for company in company_list:
	company_index = 0
	print(f'{company}:')
	company_info = rs.find_instrument_data(company)
	if len(company_info) > 1:
		for i, info in enumerate(company_info):
			# find industry
			company_fundamentals = rs.stocks.get_fundamentals(info['symbol'])
			company_sector = company_fundamentals[0]['industry']
			print(f'  {company}: {info["symbol"]} - {company_sector}')
			if 'pharma' in company_sector.lower():
				company_index = i
				break
	company_symbol = company_info[company_index]['symbol']
	# get market cap and sector
	company_fundamentals = rs.stocks.get_fundamentals(company_symbol)
	company_market_cap = int(float(company_fundamentals[0]['market_cap']))
	company_sector = company_fundamentals[0]['industry']
	company_employee = company_fundamentals[0]['num_employees']
	print(f'  {company}: {company_symbol} - {company_sector}')
	print(f'    Market Cap - ${company_market_cap}')
	print(f'    Employee Count - {company_employee}')

Pfizer:
Found 1 results
  Pfizer: PFE - Pharmaceuticals: Major
    Market Cap - $163933961403
    Employee Count - 88000
Amgen:
Found 1 results
  Amgen: AMGN - Pharmaceuticals: Major
    Market Cap - $177511705850
    Employee Count - 26700
Novartis:
Found 1 results
  Novartis: NVS - Pharmaceuticals: Major
    Market Cap - $243747962844
    Employee Count - 76057
Eli Lilly:
Found 1 results
  Eli Lilly: LLY - Pharmaceuticals: Major
    Market Cap - $904662398008
    Employee Count - 43000
Gilead:
Found 1 results
  Gilead: GILD - Biotechnology
    Market Cap - $88555252082
    Employee Count - 18000
Abbvie:
Found 1 results
  Abbvie: ABBV - Pharmaceuticals: Major
    Market Cap - $297937586400
    Employee Count - 50000
Biogen:
Found 1 results
  Biogen: BIIB - Pharmaceuticals: Major
    Market Cap - $32902010060
    Employee Count - 7570
Regeneron:
Found 1 results
  Regeneron: REGN - Pharmaceuticals: Major
    Market Cap - $120628861118
    Employee Count - 13450
Vertex:
Found 3 results
 

***
## TO DO





## Correlation between tickers

Also consider reading this paper: [Life time of correlation between stocks prices on established and emerging markets](https://arxiv.org/pdf/1105.6272.pdf)

> **Abstract:** The correlation coefficient between stocks depends on price history and
includes information on hierarchical structure in financial markets. It is
useful for portfolio selection and estimation of risk. I introduce the Life
Time of Correlation between stocks prices to know how far we should
investigate the price history to obtain the optimal durability of correlation. I
carry out my research on emerging (Poland) and established markets (in the
USA, Great Britain and Germany). Other methods, including the Minimum
Spanning Trees, tree half-life, decomposition of correlations and the Epps
effect are also discussed.

***
## Interactive Historical Data

In [19]:
import pytz
from datetime import datetime, timedelta

nytz = pytz.timezone('America/New_York')

def plot_ticker(ticker,timescale):
	buy_color="green"
	sell_color="red"

	if timescale=='long':
	# Get historical data
		ticker_data = rs.stocks.get_stock_historicals(ticker, interval='day', span='5year')
		df = pd.DataFrame(ticker_data)
		df.begins_at = df.begins_at.map(lambda x: datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ"))
		df.begins_at = df.begins_at.map(lambda x: x+timedelta(hours=12))

	elif timescale=='short':
		ticker_data = rs.stocks.get_stock_historicals(ticker, interval='5minute', span='day', bounds='extended')
		# Convert timezones
		df = pd.DataFrame(ticker_data)
		df.begins_at = pd.to_datetime(df.begins_at)
		df.begins_at = df['begins_at'].dt.tz_convert('America/New_York')
	
	# Reformat names
	df = df.rename(columns={'begins_at':'date',
							'open_price':'open',
							'high_price':'high',
							'low_price':'low',
							'close_price':'close'})

	# Set colors
	increasing_color = '#17BECF'
	decreasing_color = '#7F7F7F'
	# create volume chart colors
	colors = []
	for i in range(len(df.close)):
		if i != 0:
			if df.close[i] > df.close[i-1]:
				colors.append(increasing_color)
			else:
				colors.append(decreasing_color)
		else:
			colors.append(decreasing_color)

	# Initial candlestick chart
	layout = dict()
	fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
				vertical_spacing=0.01, 
				row_width=[0.2, 0.8])
	# OHLC
	fig.add_trace(go.Candlestick(x=df.date,
								open=df.open,
								high=df.high,
								low=df.low,
								close=df.close,
								increasing_line_color=increasing_color,
								decreasing_line_color=decreasing_color ,
								showlegend=False
								), row=1, col=1)
	# Volume
	fig.add_trace(go.Bar(x=df.date, 
						y=df.volume, 
						showlegend=True, 
						name='Volume',
						marker=dict(color=colors)
						), row=2, col=1)
	if timescale=='long':
		# MA50
		df['MA50'] = df.close.rolling(50).mean()
		fig.add_trace(go.Scatter(x=df.date, 
								y=df.MA50, 
								line=dict(color='#b300ff', width=1),
								name='MA50',
								showlegend=True
								), row=1, col=1)
		# MA200
		df['MA200'] = df.close.rolling(200).mean()
		fig.add_trace(go.Scatter(x=df.date, 
								y=df.MA200, 
								line=dict(color='#5900ff', width=1),
								name='MA200'
								), row=1, col=1)

	# Add individual orders to plot
	# orders = rs.find_stock_orders(symbol=holding)
	'''
	order_history = get_order_history(ticker)
	orders = order_history[ticker]['orders']
	if orders:
	for order in orders:      
		if order['side'] == 'buy':
		fig.add_annotation(x=order["timestamp"],
							y=float(order["price"]),
							text=str(order["quantity"])+'sh=*$'+str(float(order["price"])),
							showarrow=True,
							arrowhead=1,
							arrowcolor=buy_color,
							align='right',
							font=dict(color=buy_color,size=6))
		elif order['side'] == 'sell':
		fig.add_annotation(x=order["timestamp"],
							y=float(order["price"]),
							text=str(order["quantity"])+'sh=*$'+str(float(order["price"])),
							showarrow=True,
							arrowhead=1,
							arrowcolor=sell_color,
							align='right',
							font=dict(color=sell_color,size=6))
	# Average buy price
	avg_buy_price = order_history[ticker]['total']['avg_cost']
	print(f'Average buy price: ${round(avg_buy_price,2)}')
	fig.add_shape(type='line', xref='paper',yref='y',
					x0=0, x1=1, 
					y0=avg_buy_price, y1=avg_buy_price,
					line=dict(width=0.5, color='black')
					)
	fig.add_annotation(xref='paper',yref='y',
						x=0, y=avg_buy_price, 
						text='Average buy price: $'+str(round(avg_buy_price,2)),
						showarrow=False,
						align='right',
						font=dict(color='black',size=9)
						)
	'''
	# # Add range buttons
	fig.update_layout(
		xaxis=dict(
			rangeselector=dict(
				buttons=list([
					dict(count=1,
						 label="1d",
						 step="day",
						 stepmode="backward"),
					dict(count=7,
						 label="1w",
						 step="day",
						 stepmode="backward"),
					dict(count=1,
						label="1m",
						step="month",
						stepmode="backward"),
					dict(count=3,
						label="3m",
						step="month",
						stepmode="backward"),
					dict(count=6,
						label="6m",
						step="month",
						stepmode="backward"),
					dict(count=1,
						label="YTD",
						step="year",
						stepmode="todate"),
					dict(count=1,
						label="1y",
						step="year",
						stepmode="backward"),
					dict(step="all")
				])
			),
			type="date"
		),
		title=ticker,
		yaxis_title='Price (adjusted)'
	)
	fig.update_xaxes(type='date', row=2, col=1)
	fig.update_yaxes(title='Volume', row=2, col=1)
	fig.update(layout_xaxis_rangeslider_visible=False)
	fig.show()

In [21]:
plot_ticker('AGIO', 'long')

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [4]:
all_biopharmaceuticals = rs.markets.get_all_stocks_from_market_tag(tag='biopharmaceutical')
biopharma_df = pd.DataFrame()
for stock in all_biopharmaceuticals:
	stock_ticker = stock['symbol']
	# get name and market cap
	company_info = rs.find_instrument_data(stock_ticker)
	try:
		stock_name = company_info[0]['simple_name']
	except:
		stock_name = 'NA'
	stock_info = rs.stocks.get_fundamentals(stock_ticker)
	try:
		stock_market_cap = int(float(stock_info[0]['market_cap']))
	except:
		stock_market_cap = 'NA'
	print(f'{stock_name}: {stock_ticker} - ${stock_market_cap}')
	# insert all company info and stock info into database
	stock_dict = {'name':stock_name, 'ticker':stock_ticker, 'market_cap':stock_market_cap}
	stock_df = pd.DataFrame(stock_dict, index=[0])
	biopharma_df = pd.concat([biopharma_df, stock_df], ignore_index=True)

Found 1 results
Phathom Pharmaceuticals: PHAT - $676081560
Found 1 results
Matinas BioPharma: MTNB - $41368961
Found 1 results
BioLineRx: BLRX - $61332525
Found 1 results
Bristol-Myers Squibb: BMY - $97293101340
Found 1 results
ADMA Biologics: ADMA - $2858207423
Found 1 results
Apellis Pharmaceuticals: APLS - $4337620840
Found 1 results
Alnylam Pharmaceuticals: ALNY - $33125092500
Found 1 results
Intra-Cellular Therapies: ITCI - $8251214125
Found 1 results
CSL Limited: CSLLY - $96789283449
Found 1 results
AbbVie: ABBV - $334941392250
Found 1 results
Denali Therapeutics: DNLI - $3354877164
Found 4 results
Exelixis: EXEL - $6979380280
Found 1 results
Dynavax: DVAX - $1447665520
No results found for that keyword
NA: SRNE - $4685889
Found 1 results
Omeros: OMER - $289721000
Found 1 results
Cytokinetics: CYTK - $6836514560
Found 1 results
Theravance Biopharma: TBPH - $477821414
Found 1 results
DiaMedica Therapeutics Inc. Common Stock: DMAC - $148957024
Found 1 results
Incyte: INCY - $126796

In [40]:
'''
[{'open': '30.738000',
'high': '30.810000',
'low': '29.910000',
'volume': '14833541.000000',
'overnight_volume': '0.000000',
'bounds': 'regular',
'market_date': '2024-08-01',
'average_volume_2_weeks': '42939487.400000',
'average_volume': '42939487.400000',
'average_volume_30_days': '35502187.772700',
'high_52_weeks': '37.186000',
'high_52_weeks_date': '2023-08-21',
'dividend_yield': '5.435490',
'float': '5664142033.120000',
'low_52_weeks': '25.200000',
'low_52_weeks_date': '2024-04-26',
'market_cap': '172942708105.157379',
'pb_ratio': '1.875450',
'pe_ratio': '-66.220000',
'shares_outstanding': '5666592663.268557',
'description': 'Pfizer Inc. is a research-based global biopharmaceutical company, which engages in the discovery, development, manufacture, marketing, sales and distribution of biopharmaceutical products worldwide. The firm works across developed and emerging markets to advance wellness, prevention, treatments and cures that challenge the most feared diseases. The company was founded by Charles Pfizer Sr. and Charles Erhart in 1849 and is headquartered in New York, NY.',
'instrument': 'https://api.robinhood.com/instruments/6ec6c70e-d686-4d73-b5a8-74fec96aca0e/',
'ceo': 'Albert Bourla',
'headquarters_city': 'New York',
'headquarters_state': 'New York',
'sector': 'Health Technology',
'industry': 'Pharmaceuticals: Major',
'num_employees': 88000,
'year_founded': 1849,
'payable_date': '2024-09-03',
'ex_dividend_date': '2024-07-26',
'financial_status_indicator': 'CA0',
'financial_status_description': '',
'symbol': 'PFE'}]
'''

def get_stock_data(stock_ticker, info_type='fundamentals'):
	'''
	get_stock_data returns the requested fields for the stock
	'''
	if info_type == 'instrument_data':
		stock_info = rs.find_instrument_data(stock_ticker)
	if info_type == 'fundamentals':
		stock_info = rs.stocks.get_fundamentals(stock_ticker)
	stock_dict = {}
	if not stock_info[0]:
		return stock_dict
	for field in stock_info[0].keys():
		try:
			stock_dict[field] = stock_info[0][field]
		except:
			stock_dict[field] = 'NA'
	return stock_dict

def get_all_healthcare_stocks():
	'''
	get_all_healthcare_stocks returns all stocks that are tagged
	with healthcare, biopharmaceutical, biotechnology, or pharmaceutical
	'''
	all_healthcare_stocks = []
	healthcare_terms = ['healthcare', 'biopharmaceutical', 'biotechnology', 'pharmaceutical']
	for term in healthcare_terms:
		stocks = rs.markets.get_all_stocks_from_market_tag(tag=term)
		all_healthcare_stocks.extend(stocks)

	healthcare_df = pd.DataFrame()
	for stock in all_healthcare_stocks:
		stock_ticker = stock['symbol']
		# get name and market cap
		company_dict = get_stock_data(stock_ticker, 'instrument_data')
		stock_dict = get_stock_data(stock_ticker, 'fundamentals')
		print(f'{stock_name}: {stock_ticker} - ${stock_dict["market_cap"]}')
		# insert all company info and stock info into database
		columns = list(company_dict.keys()) + list(stock_dict.keys())
		values = list(company_dict.values()) + list(stock_dict.values())
		stock_dict = dict(zip(columns, values))
		stock_df = pd.DataFrame(stock_dict, index=[0])
		if len(healthcare_df) == 0 or stock_ticker not in healthcare_df['symbol'].values:
			healthcare_df = pd.concat([healthcare_df, stock_df], ignore_index=True)
	return healthcare_df

healthcare_df = get_all_healthcare_stocks()

Found 1 results
NA: WEX - $7456342584.815792
Found 2 results
NA: UNICY - $19912337804.081337
Found 1 results
NA: SRTS - $102782229.000000
Found 1 results
NA: MCK - $80999331060.000000
Found 1 results
NA: STRM - $30394700.000000
Found 7 results
NA: UG - $49802428.800000
Found 1 results
NA: PTH - $144417000.000000
Found 1 results
NA: CTRE - $3846389980.498884
Found 1 results
NA: ABM - $3441643115.111025
Found 9 results
NA: HEAR - $299166136.143750
Found 1 results
NA: ICUI - $3054765420.000000
No results found for that keyword
NA: MDRX - $1044108000.000000
Found 2 results
NA: NVO - $595643184105.054321
Found 7 results
NA: PRA - $635712790.200000
Found 3 results
NA: CAE - $5577622923.290777
Found 1 results
NA: JNJ - $382679527755.995789
Found 3 results
NA: MBIN - $2016868003.724454
Found 10 results
NA: COR - $48231482640.000000
Found 1 results
NA: FXH - $1274898219.810000
Found 1 results
NA: HAE - $4522087625.000000
Found 1 results
NA: HELE - $1298990246.023007
Found 1 results
NA: HSIC - $

In [33]:
healthcare_df

,name,ticker,open,high,low,volume,overnight_volume,bounds,market_date,average_volume_2_weeks,...,headquarters_state,sector,industry,num_employees,year_founded,payable_date,ex_dividend_date,financial_status_indicator,financial_status_description,symbol
0,WEX,WEX,183.315000,184.920000,180.500000,97964.000000,0.000000,regular,2024-08-01,478751.700000,...,Maine,Commercial Services,Miscellaneous Commercial Services,7200,1983,None,None,CA0,,WEX
1,Unicharm,UNICY,6.200000,6.454000,6.200000,33153.000000,0.000000,regular,2024-08-01,113705.900000,...,Tokyo,Consumer Non-Durables,Household/Personal Care,16223,1961,2024-03-18,2023-12-28,None,None,UNICY
2,Sensus,SRTS,6.370000,6.440000,6.210000,96938.000000,0.000000,regular,2024-08-01,181794.400000,...,Florida,Health Technology,Medical Specialties,35,2010,None,None,CC0,,SRTS
3,McKesson,MCK,620.980000,627.310000,620.625000,220559.000000,0.000000,regular,2024-08-01,654055.900000,...,Texas,Distribution Services,Medical Distributors,51000,1833,2024-07-01,2024-06-03,CA0,,MCK
4,Streamline Health Solutions,STRM,0.500000,0.500000,0.465000,53187.000000,0.000000,regular,2024-08-01,24158.100000,...,Georgia,Technology Services,Information Technology Services,77,1989,None,None,CC4,Noncompliant,STRM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,"Harrow, Inc.",HROW,25.810000,26.000000,24.760000,121472.000000,0.000000,regular,2024-08-01,311144.200000,...,Tennessee,Health Technology,Pharmaceuticals: Major,315,2006,None,None,CC0,,HROW
521,Alcon,ALC,93.415000,94.190000,92.780000,191343.000000,0.000000,regular,2024-08-01,904058.600000,...,Geneve (Geneva),Health Technology,Medical Specialties,25315,1945,2024-05-16,2024-05-14,CA0,,ALC
522,Kirin Holdings,KNBWY,13.524500,13.690000,13.520000,4372.000000,0.000000,regular,2024-08-01,44801.700000,...,Tokyo,Consumer Non-Durables,Beverages: Alcoholic,30183,1907,2024-04-05,2023-12-28,None,None,KNBWY
523,Ingredion,INGR,124.990000,125.465000,122.210000,70928.000000,0.000000,regular,2024-08-01,290203.300000,...,Illinois,Consumer Non-Durables,Food: Specialty/Candy,11600,1906,2024-07-23,2024-07-01,CA0,,INGR


In [39]:
company_info = rs.find_instrument_data('MDRX')
company_info[0]

No results found for that keyword


In [10]:
stock_info = rs.stocks.get_fundamentals('PFE')
stock_info[0].keys()

[{'open': '30.738000',
  'high': '30.810000',
  'low': '29.910000',
  'volume': '14833541.000000',
  'overnight_volume': '0.000000',
  'bounds': 'regular',
  'market_date': '2024-08-01',
  'average_volume_2_weeks': '42939487.400000',
  'average_volume': '42939487.400000',
  'average_volume_30_days': '35502187.772700',
  'high_52_weeks': '37.186000',
  'high_52_weeks_date': '2023-08-21',
  'dividend_yield': '5.435490',
  'float': '5664142033.120000',
  'low_52_weeks': '25.200000',
  'low_52_weeks_date': '2024-04-26',
  'market_cap': '172942708105.157379',
  'pb_ratio': '1.875450',
  'pe_ratio': '-66.220000',
  'shares_outstanding': '5666592663.268557',
  'description': 'Pfizer Inc. is a research-based global biopharmaceutical company, which engages in the discovery, development, manufacture, marketing, sales and distribution of biopharmaceutical products worldwide. The firm works across developed and emerging markets to advance wellness, prevention, treatments and cures that challenge t

In [26]:
stock_info[0].keys()

dict_keys(['open', 'high', 'low', 'volume', 'overnight_volume', 'bounds', 'market_date', 'average_volume_2_weeks', 'average_volume', 'average_volume_30_days', 'high_52_weeks', 'high_52_weeks_date', 'dividend_yield', 'float', 'low_52_weeks', 'low_52_weeks_date', 'market_cap', 'pb_ratio', 'pe_ratio', 'shares_outstanding', 'description', 'instrument', 'ceo', 'headquarters_city', 'headquarters_state', 'sector', 'industry', 'num_employees', 'year_founded', 'payable_date', 'ex_dividend_date', 'financial_status_indicator', 'financial_status_description', 'symbol'])

[{'ask_price': '181.070000',
  'ask_size': 8,
  'venue_ask_time': '2024-08-01T15:34:25.879905662Z',
  'bid_price': '180.580000',
  'bid_size': 25,
  'venue_bid_time': '2024-08-01T15:34:25.879905662Z',
  'last_trade_price': '180.810000',
  'venue_last_trade_time': '2024-08-01T15:34:07.713558246Z',
  'last_extended_hours_trade_price': None,
  'last_non_reg_trade_price': None,
  'venue_last_non_reg_trade_time': None,
  'previous_close': '183.450000',
  'adjusted_previous_close': '183.450000',
  'previous_close_date': '2024-07-31',
  'symbol': 'WEX',
  'trading_halted': False,
  'has_traded': True,
  'last_trade_price_source': 'nls',
  'last_non_reg_trade_price_source': '',
  'updated_at': '2024-08-01T15:34:25Z',
  'instrument': 'https://api.robinhood.com/instruments/5ea7992b-e531-495d-a16c-7db8a7ec637d/',
  'instrument_id': '5ea7992b-e531-495d-a16c-7db8a7ec637d',
  'state': 'active'},
 {'ask_price': '6.440000',
  'ask_size': 2400,
  'venue_ask_time': '2024-08-01T15:34:31.347Z',
  'bid_pri